In [1]:
! pip install transformers sentencepiece

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached transformers-4.16.2-py3-none-any.whl (3.5 MB)
  Using cached sentencepiece-0.1.96-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
     |████████████████████████████████| 6.8 MB 1.3 MB/s eta 0:00:01
  Using cached sacremoses-0.0.47-py2.py3-none-any.whl (895 kB)
  Using cached huggingface_hub-0.4.0-py3-none-any.whl (67 kB)


In [2]:
!nvidia-smi

Thu Feb 24 09:52:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| 23%   31C    P5    23W / 160W |    557MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
# DECLARE HOW MANY GPUS YOU WISH TO USE. 
# KAGGLE ONLY HAS 1, BUT OFFLINE, YOU CAN USE MORE
os.environ["CUDA_VISIBLE_DEVICES"]="0" #0,1,2,3 for four gpu

MODEL_NAME = 'allenai/longformer-base-4096'

In [3]:
import numpy as np

In [4]:
from torch import cuda
config = {'model_name': MODEL_NAME,   
         'max_length': 1024,
         'train_batch_size': 12,
         'valid_batch_size':4,
         'epochs':2,
         'learning_rates': np.array([5e-5, 2.5e-5, 5e-5, 2.5e-6, 2.5e-6])/4,
         'max_grad_norm':10,
         'device': 'cuda' if cuda.is_available() else 'cpu'}

In [5]:
from data_helpers import *
import inference
import evaluation
import model_helpers

import numpy as np, os 
import pandas as pd, gc 
from tqdm import tqdm

In [6]:
model, tokenizer = model_helpers.load_model(config['model_name'], config['model_name'])

Downloading: 100%|██████████| 694/694 [00:00<00:00, 281kB/s]
Downloading: 100%|██████████| 878k/878k [00:00<00:00, 1.11MB/s] 
Downloading: 100%|██████████| 446k/446k [00:00<00:00, 882kB/s] 
Downloading: 100%|██████████| 1.29M/1.29M [00:00<00:00, 1.68MB/s]
file allenai/longformer-base-4096/config.json not found


OSError: Can't load config for 'allenai/longformer-base-4096'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'allenai/longformer-base-4096' is the correct path to a directory containing a config.json file

In [ ]:
train_params, test_params, output_labels, ids_to_labels, labels_to_ids = model_helpers.get_params(config)
training_loader, testing_loader, test_texts_loader, test_dataset = model_helpers.get_train_data(tokenizer)

In [ ]:
from sklearn.metrics import accuracy_score

def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    #tr_preds, tr_labels = [], []
    
    # put model in training mode
    model.train()
    loop = tqdm(training_loader, leave=True)
    idx = 0
    for batch in loop:
        
        optimizer.zero_grad()

        ids = batch['input_ids'].to(config['device'], dtype = torch.long)
        mask = batch['attention_mask'].to(config['device'], dtype = torch.long)
        labels = batch['labels'].to(config['device'], dtype = torch.long)

        loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels,
                               return_dict=False)
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        #if idx % 200==0:
        #    loss_step = tr_loss/nb_tr_steps
        #    print(f"Training loss after {idx:04d} training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        #tr_labels.extend(labels)
        #tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=config['max_grad_norm']
        )
        
        # backward pass
        #loss.backward()
        scaler.scale(loss).backward()
        #optimizer.step()
        scaler.step(optimizer)
        scaler.update()

        idx+=1
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item(), accuracy=tmp_tr_accuracy)


    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [ ]:
# CREATE MODEL
from transformers import AdamW
from transformers import get_cosine_schedule_with_warmup

config_model = AutoConfig.from_pretrained(DOWNLOADED_MODEL_PATH+'/config.json') 
model = AutoModelForTokenClassification.from_pretrained(
                   DOWNLOADED_MODEL_PATH+'/pytorch_model.bin',config=config_model)
model.to(config['device'])
optimizer = torch.optim.Adam(params=model.parameters(), lr=config['learning_rates'][0])

Some weights of the model checkpoint at /content/longformer-base-pretrained/pytorch_model.bin were not used when initializing LongformerForTokenClassification: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing LongformerForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForTokenClassification were not initialized from the model checkpoint at /content/longformer-base-pretrained/pytorch_model.bin and are newly initi

In [ ]:
model.load_state_dict(torch.load("more_1.pt"))

<All keys matched successfully>

In [ ]:
config['epochs'] = 1
config['learning_rates'] = [2.5e-5/2]

In [ ]:
scaler = torch.cuda.amp.GradScaler()

In [ ]:
for epoch in range(config['epochs']):
    
    print(f"### Training epoch: {epoch + 1}")
    for g in optimizer.param_groups: 
        g['lr'] = config['learning_rates'][epoch]
    lr = optimizer.param_groups[0]['lr']
    print(f'### LR = {lr}\n')
    
    train(epoch)
    torch.cuda.empty_cache()
    gc.collect()
    model.eval()
    eval_score = evaluate_model()
    model.train()
    torch.cuda.empty_cache()
    gc.collect()
    print(f"Eval Score = {eval_score}")
    torch.save(model.state_dict(), f'more_{epoch}.pt')


### Training epoch: 1
### LR = 1.25e-05



Epoch 0: 100%|██████████| 2565/2565 [27:51<00:00,  1.53it/s, accuracy=0.843, loss=0.462]


Training loss epoch: 0.34133102656805026
Training accuracy epoch: 0.8850766515891456
HERE
1894
HERE
Eval Score = 0.720791470226404


In [ ]:
evaluate_model()

HERE
929
HERE


0.5403163614491813

In [ ]:
config['epochs'] = 2
config['learning_rates'] = np.array([2.5e-5, 2.5e-6, 2.5e-5, 2.5e-5, 2.5e-6])/4

In [ ]:
for epoch in range(config['epochs']):
    
    print(f"### Training epoch: {epoch + 1}")
    for g in optimizer.param_groups: 
        g['lr'] = config['learning_rates'][epoch]
    lr = optimizer.param_groups[0]['lr']
    print(f'### LR = {lr}\n')
    
    train(epoch)
    torch.cuda.empty_cache()
    gc.collect()
    model.eval()
    eval_score = evaluate_model()
    model.train()
    torch.cuda.empty_cache()
    gc.collect()
    print(f"Eval Score = {eval_score}")
    torch.save(model.state_dict(), f'more_{epoch}.pt')

### Training epoch: 1
### LR = 6.25e-06



Epoch 0: 100%|██████████| 1287/1287 [13:59<00:00,  1.53it/s, accuracy=0.842, loss=0.501]


Training loss epoch: 0.4530334774037156
Training accuracy epoch: 0.8385425406829212
HERE
1357
HERE
Eval Score = 0.6482201487159749
### Training epoch: 2
### LR = 6.25e-07



Epoch 1: 100%|██████████| 1287/1287 [13:59<00:00,  1.53it/s, accuracy=0.796, loss=0.476]


Training loss epoch: 0.41444853719297825
Training accuracy epoch: 0.851258822876064
HERE
1366
HERE
Eval Score = 0.6420754234517881


In [ ]:
! cp more_0.pt /content/drive/MyDrive/feedback-kaggle/
! cp more_1.pt /content/drive/MyDrive/feedback-kaggle/

In [ ]:
! mkdir 
! cp longformer_on_training_0.pt longformer-base-pretrained/
! cp longformer_on_training_1.pt longformer-base-pretrained/
! cp longformer_on_training_2.pt longformer-base-pretrained/
!zip -r longformer_base_on_pr_and_train.zip model/
! cp longformer_base_on_pr_and_train.zip /content/drive/MyDrive/feedback-kaggle/

  adding: model/ (stored 0%)
  adding: model/special_tokens_map.json (deflated 50%)
  adding: model/config.json (deflated 63%)
  adding: model/longformer_1.pt (deflated 13%)
  adding: model/merges.txt (deflated 53%)
  adding: model/tokenizer_config.json (deflated 47%)
  adding: model/vocab.json (deflated 59%)
  adding: model/longformer_4.pt (deflated 13%)
  adding: model/tokenizer.json (deflated 59%)
  adding: model/longformer_0.pt (deflated 13%)
  adding: model/longformer_2.pt (deflated 13%)
  adding: model/longformer_3.pt (deflated 13%)


In [ ]:
def inference(batch):
                
    # MOVE BATCH TO GPU AND INFER
    ids = batch["input_ids"].to(config['device'])
    mask = batch["attention_mask"].to(config['device'])
    outputs = model(ids, attention_mask=mask, return_dict=False)
    all_preds = torch.argmax(outputs[0], axis=-1).cpu().numpy() 

    # INTERATE THROUGH EACH TEXT AND GET PRED
    predictions = []
    for k,text_preds in enumerate(all_preds):
        token_preds = [ids_to_labels[i] for i in text_preds]

        prediction = []
        word_ids = batch['wids'][k].numpy()  
        previous_word_idx = -1
        for idx,word_idx in enumerate(word_ids):                            
            if word_idx == -1:
                pass
            elif word_idx != previous_word_idx:              
                prediction.append(token_preds[idx])
                previous_word_idx = word_idx
        predictions.append(prediction)
    
    return predictions

In [ ]:
# https://www.kaggle.com/zzy990106/pytorch-ner-infer
# code has been modified from original
def get_predictions(df=test_dataset, loader=testing_loader):
    
    # put model in training mode
    model.eval()
    
    # GET WORD LABEL PREDICTIONS
    y_pred2 = []
    for batch in loader:
        try:
            labels = inference(batch)
            y_pred2.extend(labels)
        except Exception as e:
            pass
    print("HERE")

    final_preds2 = []
    for i in range(len(df)):
        idx = df.id.values[i]
        #pred = [x.replace('B-','').replace('I-','') for x in y_pred2[i]]
        pred = y_pred2[i] # Leave "B" and "I"
        preds = []
        j = 0
        while j < len(pred):
            cls = pred[j]
            if cls == 'O': j += 1
            else: cls = cls.replace('B','I') # spans start with B
            end = j + 1
            while end < len(pred) and pred[end] == cls:
                end += 1
            
            if cls != 'O' and cls != '' and end - j > 7:
                final_preds2.append((idx, cls.replace('I-',''),
                                    ' '.join(map(str, list(range(j, end))))))
        
            j = end
    print(len(final_preds2))
    print("HERE")
        
    oof = pd.DataFrame(final_preds2)
    oof.columns = ['id','class','predictionstring']

    return oof

In [ ]:
# from Rob Mulla @robikscube
# https://www.kaggle.com/robikscube/student-writing-competition-twitch
def calc_overlap(row):
    """
    Calculates the overlap between prediction and
    ground truth and overlap percentages used for determining
    true positives.
    """
    set_pred = set(row.predictionstring_pred.split(' '))
    set_gt = set(row.predictionstring_gt.split(' '))
    # Length of each and intersection
    len_gt = len(set_gt)
    len_pred = len(set_pred)
    inter = len(set_gt.intersection(set_pred))
    overlap_1 = inter / len_gt
    overlap_2 = inter/ len_pred
    return [overlap_1, overlap_2]



In [ ]:
# from Rob Mulla @robikscube
# https://www.kaggle.com/robikscube/student-writing-competition-twitch
def calc_overlap(row):
    """
    Calculates the overlap between prediction and
    ground truth and overlap percentages used for determining
    true positives.
    """
    set_pred = set(row.predictionstring_pred.split(' '))
    set_gt = set(row.predictionstring_gt.split(' '))
    # Length of each and intersection
    len_gt = len(set_gt)
    len_pred = len(set_pred)
    inter = len(set_gt.intersection(set_pred))
    overlap_1 = inter / len_gt
    overlap_2 = inter/ len_pred
    return [overlap_1, overlap_2]


def score_feedback_comp(pred_df, gt_df):
    """
    A function that scores for the kaggle
        Student Writing Competition
        
    Uses the steps in the evaluation page here:
        https://www.kaggle.com/c/feedback-prize-2021/overview/evaluation
    """
    gt_df = gt_df[['id','discourse_type','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df = pred_df[['id','class','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df['pred_id'] = pred_df.index
    gt_df['gt_id'] = gt_df.index
    # Step 1. all ground truths and predictions for a given class are compared.
    joined = pred_df.merge(gt_df,
                           left_on=['id','class'],
                           right_on=['id','discourse_type'],
                           how='outer',
                           suffixes=('_pred','_gt')
                          )
    joined['predictionstring_gt'] = joined['predictionstring_gt'].fillna(' ')
    joined['predictionstring_pred'] = joined['predictionstring_pred'].fillna(' ')

    joined['overlaps'] = joined.apply(calc_overlap, axis=1)

    # 2. If the overlap between the ground truth and prediction is >= 0.5, 
    # and the overlap between the prediction and the ground truth >= 0.5,
    # the prediction is a match and considered a true positive.
    # If multiple matches exist, the match with the highest pair of overlaps is taken.
    joined['overlap1'] = joined['overlaps'].apply(lambda x: eval(str(x))[0])
    joined['overlap2'] = joined['overlaps'].apply(lambda x: eval(str(x))[1])


    joined['potential_TP'] = (joined['overlap1'] >= 0.5) & (joined['overlap2'] >= 0.5)
    joined['max_overlap'] = joined[['overlap1','overlap2']].max(axis=1)
    tp_pred_ids = joined.query('potential_TP') \
        .sort_values('max_overlap', ascending=False) \
        .groupby(['id','predictionstring_gt']).first()['pred_id'].values

    # 3. Any unmatched ground truths are false negatives
    # and any unmatched predictions are false positives.
    fp_pred_ids = [p for p in joined['pred_id'].unique() if p not in tp_pred_ids]

    matched_gt_ids = joined.query('potential_TP')['gt_id'].unique()
    unmatched_gt_ids = [c for c in joined['gt_id'].unique() if c not in matched_gt_ids]

    # Get numbers of each type
    TP = len(tp_pred_ids)
    FP = len(fp_pred_ids)
    FN = len(unmatched_gt_ids)
    #calc microf1
    my_f1_score = TP / (TP + 0.5*(FP+FN))
    return my_f1_score

In [ ]:
def evaluate_model():
    valid = train_df.loc[train_df['id'].isin(IDS[valid_idx])]

    # OOF PREDICTIONS

    testing_loader = DataLoader(testing_set, **test_params)
    oof = get_predictions(test_dataset, testing_loader)

    # COMPUTE F1 SCORE
    f1s = []
    CLASSES = oof['class'].unique()
    for c in CLASSES:
        pred_df = oof.loc[oof['class']==c].copy()
        gt_df = valid.loc[valid['discourse_type']==c].copy()
        f1 = score_feedback_comp(pred_df, gt_df)
        f1s.append(f1)
    return np.mean(f1s)

# Longformer large after 5 epochs
Position 0.5966101694915255
Claim 0.5456521739130434
Evidence 0.6977730646871686
Concluding Statement 0.7331189710610932
Lead 0.7897435897435897
Counterclaim 0.5567010309278351
Rebuttal 0.4

Overall 0.6170855714034651

# Longformer after 6 epochs (with almost all)
Position 0.5906040268456376
Claim 0.5257270693512305
Evidence 0.6791666666666667
Concluding Statement 0.7933333333333333
Lead 0.8082901554404145
Rebuttal 0.345679012345679
Counterclaim 0.5102040816326531

Overall 0.6075720493736593

# Longformer after 5 epochs
Position 0.6382411725516323
Claim 0.5186533212010919
Evidence 0.6483357452966715
Concluding Statement 0.7983539094650206
Lead 0.7978723404255319
Counterclaim 0.4883720930232558
Rebuttal 0.35323383084577115

Overall 0.6061517732584251

#Longformer after 4 epochs
Position 0.644718792866941
Claim 0.5136384650947757
Evidence 0.6485138224901268
Concluding Statement 0.7961432506887053
Lead 0.7957894736842105
Counterclaim 0.5059760956175299
Rebuttal 0.3641160949868074

Overall 0.6098422850612994

# ROBERTA AFTER 6 EPOCHS
Position 0.6671168129642133
Claim 0.5077157139752227
Evidence 0.6000864677907479
Concluding Statement 0.6167471819645732
Lead 0.8021505376344086
Counterclaim 0.42376237623762375
Rebuttal 0.2565597667638484

Overall 0.5534484081900911